In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
batch_size = 128
latent_size = 20
hidden_size = 400
input_size = 784
learning_rate = 1e-3
num_epochs = 20

# MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

# Variational Autoencoder (VAE) model
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 2 * latent_size)  # Outputting both mean and variance
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(latent_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, input_size),
            nn.Sigmoid()  # To get pixel values between 0 and 1
        )

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        # Encoding
        latent_params = self.encoder(x)
        mu, log_var = torch.chunk(latent_params, 2, dim=-1)
        z = self.reparameterize(mu, log_var)

        # Decoding
        reconstructed_x = self.decoder(z)
        
        return reconstructed_x, mu, log_var

# Instantiate the model
model = VAE().to(device)

# Loss function - ELBO (Evidence Lower Bound)
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
batch_size = 128
latent_size = 20
hidden_size = 400
input_size = 784
learning_rate = 1e-3
num_epochs = 20

# MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)


In [4]:

# Variational Autoencoder (VAE) model
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 2 * latent_size)  # Outputting both mean and variance
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(latent_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, input_size),
            nn.Sigmoid()  # To get pixel values between 0 and 1
        )

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        # Encoding
        latent_params = self.encoder(x)
        mu, log_var = torch.chunk(latent_params, 2, dim=-1)
        z = self.reparameterize(mu, log_var)

        # Decoding
        reconstructed_x = self.decoder(z)
        
        return reconstructed_x, mu, log_var

# Instantiate the model
model = VAE().to(device)

# Loss function - ELBO (Evidence Lower Bound) with MSE loss
def loss_function(recon_x, x, mu, log_var):
    BCE = nn.functional.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')#alculate MSE loss
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, log_var = model(data.view(-1, 784))
        loss = loss_function(recon_batch, data, mu, log_var)
        loss.backward()
        total_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Epoch [{}/{}], Batch [{}/{}], Loss: {:.3f}'.format(epoch+1, num_epochs, batch_idx, len(train_loader), loss.item() / len(data)))
    print('====> Epoch {} Average Loss: {:.4f}'.format(epoch+1, total_loss / len(train_loader.dataset)))


# Optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, log_var = model(data.view(-1, 784))
        loss = loss_function(recon_batch, data, mu, log_var)
        loss.backward()
        total_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Epoch [{}/{}], Batch [{}/{}], Loss: {:.3f}'.format(epoch+1, num_epochs, batch_idx, len(train_loader), loss.item() / len(data)))
    print('====> Epoch {} Average Loss: {:.4f}'.format(epoch+1, total_loss / len(train_loader.dataset)))


RuntimeError: all elements of target should be between 0 and 1